<a href="https://colab.research.google.com/github/sandeepkundala/Machine-Learning/blob/master/TrafficClassification3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install xgboost

# import all necessary packages
import matplotlib.pyplot as plt # plotting
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link = 'https://drive.google.com/open?id=1ts1QJMNy6vuuVAu-Lq-n_lAcVzFb93FN'
fluff, id = link.split('=')
print(id)

1ts1QJMNy6vuuVAu-Lq-n_lAcVzFb93FN


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('NIMS-normalized.arff')

In [0]:
from scipy.io import arff

data = arff.loadarff('NIMS-normalized.arff')
df = pd.DataFrame(data[0])

In [6]:
df = df[:50000]
labels = []
for i in df['class'].unique().tolist():
    labels.append(str(i))
df.drop(['class'], axis = 1)
df['Class_Factor'] = df['class'].factorize()[0]
df.head()

,min_fpktl,mean_fpktl,max_fpktl,std_fpktl,min_bpktl,mean_bpktl,max_bpktl,std_bpktl,min_fiat,mean_fiat,max_fiat,std_fiat,min_biat,mean_biat,max_biat,std_biat,duration,proto,total_fpackets,total_fvolume,total_bpackets,total_bvolume,class,Class_Factor
0,0.000000,0.000593,0.000122,0.000367,0.064062,0.032667,0.045582,0.018614,0.000212,0.022841,0.024289,0.034299,0.000106,0.011792,0.002833,0.011884,0.002152,0.0,0.000012,0.000000,0.000032,0.000002,b'TELNET',0
1,0.000000,0.000711,0.000488,0.000490,0.064062,0.040667,0.039584,0.017580,0.000191,0.012860,0.021976,0.032089,0.000124,0.009288,0.006224,0.019656,0.000071,0.0,0.000012,0.000000,0.000034,0.000003,b'FTP',1
2,0.020202,0.022048,0.025140,0.025340,0.082812,0.594000,0.600160,0.627715,0.000089,0.010255,0.015281,0.022949,0.000347,0.009130,0.004354,0.016111,0.002042,0.0,0.000015,0.000002,0.000030,0.000039,b'HTTP',2
3,0.026936,0.002608,0.000976,0.000490,0.120313,0.114000,0.085566,0.066184,0.138378,0.138378,0.038563,0.000000,0.003322,0.003322,0.000276,0.000000,0.023441,1.0,0.000003,0.000000,0.000009,0.000002,b'DNS',3
4,0.102694,0.007231,0.003234,0.000000,0.306250,0.275333,0.253898,0.118925,0.000899,0.038567,0.070267,0.126904,0.001102,0.037950,0.020662,0.093575,0.078102,1.0,0.000022,0.000002,0.000045,0.000027,b'lime',4


In [7]:
corr_matrix = abs(df.corr(method='pearson'))
y=corr_matrix['Class_Factor'].sort_values(ascending=False)
print('Correlation between column X and column \'Class Factor\'\n',y)

Correlation between column X and column 'Class Factor'
 Class_Factor      1.000000
max_fpktl         0.874487
std_fpktl         0.874335
mean_fpktl        0.874326
total_fvolume     0.680311
duration          0.553554
proto             0.498867
max_biat          0.460839
total_bpackets    0.449449
std_biat          0.369923
max_fiat          0.353472
total_fpackets    0.352143
total_bvolume     0.301724
max_bpktl         0.291694
std_fiat          0.263235
min_fpktl         0.150464
mean_biat         0.130406
mean_bpktl        0.072702
min_biat          0.065878
std_bpktl         0.041545
min_fiat          0.031310
min_bpktl         0.023323
mean_fiat         0.001940
Name: Class_Factor, dtype: float64


In [8]:
colNames = df.columns.values
X = df.reindex(columns=colNames[:22])
Y = df['Class_Factor']
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, Y, df.index, test_size=0.2, random_state=0)
print('Data size of training set:',len(X_train))
print('Data size of testing set:',len(X_test))

Data size of training set: 40000
Data size of testing set: 10000


In [16]:
#NN Classifier 
from keras.models import Sequential
from keras.layers import Flatten, Conv1D, MaxPooling1D, Dropout, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [17]:
# Define model
model = Sequential()
model.add(Dense(100, input_dim=22, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
model.summary() #Print model Summary

W0817 23:02:25.804463 140255885637504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 23:02:25.828742 140255885637504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 23:02:25.831872 140255885637504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2300      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 7,401
Trainable params: 7,401
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile model
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["accuracy"])
             

In [23]:
# Fit Model
model.fit(X_train, y_train, epochs=10)
pred=model.predict(X_test)

Epoch 1/10
40000/40000 [==============================] - 2s 52us/step - loss: 0.4930 - acc: 0.9166
Epoch 2/10
40000/40000 [==============================] - 2s 40us/step - loss: 0.0332 - acc: 0.9754
Epoch 3/10
40000/40000 [==============================] - 2s 39us/step - loss: 0.0264 - acc: 0.9798
Epoch 4/10
40000/40000 [==============================] - 2s 38us/step - loss: 0.0221 - acc: 0.9829
Epoch 5/10
40000/40000 [==============================] - 2s 38us/step - loss: 0.0191 - acc: 0.9850
Epoch 6/10
40000/40000 [==============================] - 2s 38us/step - loss: 0.0168 - acc: 0.9871
Epoch 7/10
40000/40000 [==============================] - 2s 39us/step - loss: 0.0151 - acc: 0.9879
Epoch 8/10
40000/40000 [==============================] - 2s 39us/step - loss: 0.0141 - acc: 0.9893
Epoch 9/10
40000/40000 [==============================] - 2s 39us/step - loss: 0.0129 - acc: 0.9906
Epoch 10/10
40000/40000 [==============================] - 2s 38us/step - loss: 0.0120 - acc: 0.9910

In [24]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.16%
